<a href="https://colab.research.google.com/github/FatimaJHussain/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [24.5 kB]
Get:12 http://security.ubuntu.

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-04-18 01:14:54--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.72MB/s    in 0.2s    

2021-04-18 01:14:54 (5.72 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://uftchallenge16.s3.us-east-2.amazonaws.com/amazon_reviews_us_Wireless_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Wireless_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

### Create DataFrames to match tables

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame.    
from pyspark.sql.types import IntegerType

df=df.withColumn("customer_id", df["customer_id"].cast(IntegerType()))
df=df.withColumn("product_parent", df["product_parent"].cast(IntegerType()))
df=df.withColumn("star_rating", df["star_rating"].cast(IntegerType()))
df=df.withColumn("helpful_votes", df["helpful_votes"].cast(IntegerType()))
df=df.withColumn("total_votes", df["total_votes"].cast(IntegerType()))
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

In [6]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [7]:
# Create the vine_table. DataFrame

vine_table=df.select(['review_id','verified_purchase', 'total_votes','helpful_votes', 'vine','star_rating'])
vine_table.show()
# vine_df = df.select([])


+--------------+-----------------+-----------+-------------+----+-----------+
|     review_id|verified_purchase|total_votes|helpful_votes|vine|star_rating|
+--------------+-----------------+-----------+-------------+----+-----------+
|R3W4P9UBGNGH1U|                Y|          3|            1|   N|          2|
|R15V54KBMTQWAY|                N|          0|            0|   N|          4|
| RY8I449HNXSVF|                Y|          0|            0|   N|          5|
|R18TLJYCKJFLSR|                Y|          0|            0|   N|          5|
|R1NK26SWS53B8Q|                Y|          0|            0|   N|          5|
|R11LOHEDYJALTN|                Y|          0|            0|   N|          5|
|R3ALQVQB2P9LA7|                Y|          0|            0|   N|          4|
|R3MWLXLNO21PDQ|                Y|          0|            0|   N|          5|
|R2L15IS24CX0LI|                Y|          0|            0|   N|          5|
|R1DJ8976WPWVZU|                Y|          0|            0|   N

In [8]:
#########Deliverable 2######################
###create a new dataframe where the total_votes count is equal to or greater than 20 
vine_table_filtered=vine_table.filter(vine_table["total_votes"]>=20)
vine_table_filtered.show()


+--------------+-----------------+-----------+-------------+----+-----------+
|     review_id|verified_purchase|total_votes|helpful_votes|vine|star_rating|
+--------------+-----------------+-----------+-------------+----+-----------+
|R2WOW0TURNXB26|                Y|         59|           54|   N|          3|
|R13VL62Y2HBQ0B|                Y|         21|           15|   N|          5|
|R22G55KAPZKJQV|                Y|         21|           20|   N|          4|
|R1610PGTJS7G3N|                Y|         44|           28|   N|          2|
| RLQL04BL0QXOJ|                Y|         47|           45|   N|          4|
|R2AYJHH8WJNGAU|                Y|        104|           98|   N|          4|
|R111DJA10Y6CMU|                Y|         39|           26|   N|          5|
|R2EE2TR4MRDV0U|                Y|         59|           53|   N|          5|
| RD4A80I5JDHED|                N|         77|           69|   N|          5|
|R1GU6IYZQWQE8X|                Y|         25|           24|   N

In [9]:
#### create a new dataframe  where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
vine_table_ratio=vine_table_filtered["helpful_votes"]/vine_table_filtered["total_votes"]
vine_table_per=vine_table_filtered.filter(vine_table_ratio>=0.5)
vine_table_per.show()

+--------------+-----------------+-----------+-------------+----+-----------+
|     review_id|verified_purchase|total_votes|helpful_votes|vine|star_rating|
+--------------+-----------------+-----------+-------------+----+-----------+
|R2WOW0TURNXB26|                Y|         59|           54|   N|          3|
|R13VL62Y2HBQ0B|                Y|         21|           15|   N|          5|
|R22G55KAPZKJQV|                Y|         21|           20|   N|          4|
|R1610PGTJS7G3N|                Y|         44|           28|   N|          2|
| RLQL04BL0QXOJ|                Y|         47|           45|   N|          4|
|R2AYJHH8WJNGAU|                Y|        104|           98|   N|          4|
|R111DJA10Y6CMU|                Y|         39|           26|   N|          5|
|R2EE2TR4MRDV0U|                Y|         59|           53|   N|          5|
| RD4A80I5JDHED|                N|         77|           69|   N|          5|
|R1GU6IYZQWQE8X|                Y|         25|           24|   N

In [10]:
#### create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid)vine == 'Y'
vine_review_yes= vine_table_per.filter(vine_table_per["vine"]=="Y")
vine_review_yes.show()

+--------------+-----------------+-----------+-------------+----+-----------+
|     review_id|verified_purchase|total_votes|helpful_votes|vine|star_rating|
+--------------+-----------------+-----------+-------------+----+-----------+
|R1MAOLI5FJHAFM|                N|        261|          249|   Y|          4|
| R9PYAUDIBJVEC|                N|         22|           12|   Y|          4|
| R6V9SHMMG5M8F|                N|        110|          101|   Y|          5|
|R37PVLT6ELL5J4|                N|        209|          181|   Y|          4|
| R2FSFGWZF24V9|                N|         51|           50|   Y|          4|
|R3SRW1E8J56IGV|                N|        281|          262|   Y|          5|
| R86Z11D4CWOFM|                N|         36|           32|   Y|          4|
| RNP01HW9YISJO|                N|         23|           20|   Y|          4|
|R3KLACA6LCDZ0S|                N|         22|           21|   Y|          3|
| RZEQYOT2RE0N7|                N|         80|           75|   Y

In [11]:
# create dataframe or table that retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.
vine_review_no= vine_table_per.filter(vine_table_per["vine"]=="N")
vine_review_no.show()

+--------------+-----------------+-----------+-------------+----+-----------+
|     review_id|verified_purchase|total_votes|helpful_votes|vine|star_rating|
+--------------+-----------------+-----------+-------------+----+-----------+
|R2WOW0TURNXB26|                Y|         59|           54|   N|          3|
|R13VL62Y2HBQ0B|                Y|         21|           15|   N|          5|
|R22G55KAPZKJQV|                Y|         21|           20|   N|          4|
|R1610PGTJS7G3N|                Y|         44|           28|   N|          2|
| RLQL04BL0QXOJ|                Y|         47|           45|   N|          4|
|R2AYJHH8WJNGAU|                Y|        104|           98|   N|          4|
|R111DJA10Y6CMU|                Y|         39|           26|   N|          5|
|R2EE2TR4MRDV0U|                Y|         59|           53|   N|          5|
| RD4A80I5JDHED|                N|         77|           69|   N|          5|
|R1GU6IYZQWQE8X|                Y|         25|           24|   N

In [12]:
# total number of reviews
total_review_count= vine_table_per.count()
total_review_count

65581

In [13]:
#the number of 5-star reviews
number_5star_review=vine_table_per.filter(vine_table_per["star_rating"]==5).count()
number_5star_review

30765

In [14]:
vine_review_paid=vine_review_yes

vine_review_paid.show()

+--------------+-----------------+-----------+-------------+----+-----------+
|     review_id|verified_purchase|total_votes|helpful_votes|vine|star_rating|
+--------------+-----------------+-----------+-------------+----+-----------+
|R1MAOLI5FJHAFM|                N|        261|          249|   Y|          4|
| R9PYAUDIBJVEC|                N|         22|           12|   Y|          4|
| R6V9SHMMG5M8F|                N|        110|          101|   Y|          5|
|R37PVLT6ELL5J4|                N|        209|          181|   Y|          4|
| R2FSFGWZF24V9|                N|         51|           50|   Y|          4|
|R3SRW1E8J56IGV|                N|        281|          262|   Y|          5|
| R86Z11D4CWOFM|                N|         36|           32|   Y|          4|
| RNP01HW9YISJO|                N|         23|           20|   Y|          4|
|R3KLACA6LCDZ0S|                N|         22|           21|   Y|          3|
| RZEQYOT2RE0N7|                N|         80|           75|   Y

In [15]:

#the number of 5-star reviews for the paid reviews


review_paid_count=vine_review_paid.filter(vine_review_paid["star_rating"]==5).count()
review_paid_count

222

In [16]:
vine_review_unpaid=vine_review_no

In [17]:
#the number of 5-star reviews for the unpaid reviews
review_unpaid_count=vine_review_unpaid.filter(vine_review_unpaid["star_rating"]==5).count()
review_unpaid_count

30543

In [20]:
#the percentage of 5-star reviews for the paid reviews
total_paid_review=vine_review_yes.count()
percentage_paid_review=(review_paid_count/total_paid_review)*100
percentage_paid_review


36.215334420880914

In [21]:
#the percentage of 5-star reviews for the unpaid reviews
total_unpaid_review=vine_review_no.count()
percentage_unpaid_review=(review_unpaid_count/total_unpaid_review)*100
percentage_unpaid_review


47.01237532323606

In [22]:
#How many Vine reviews and non-Vine reviews were there?   paid
total_paid_review

613

In [23]:
#How many Vine reviews and non-Vine reviews were there?   unpaid
total_unpaid_review

64968

In [24]:
#How many Vine reviews were 5 stars? How many non-Vine reviews were 5 stars? PAID
review_paid_count

222

In [25]:
#How many Vine reviews were 5 stars? How many non-Vine reviews were 5 stars? UNPAID
review_unpaid_count

30543